zebra puzzles

In [37]:
from copy import deepcopy
from random import randint
from icecream import ic
from tqdm.auto import tqdm
from random import choice, random, sample, randrange
from math import exp

In [38]:
NUM_ITEM = 50
NUM_CATEGORIES = 10


In [39]:
ITEMS = [[(c,n) for n in range(NUM_ITEM)] for c in range(NUM_CATEGORIES)]
INCOMPATIBLE = [{randint(0,NUM_ITEM-1),randint(0,NUM_ITEM-1) , (randint(0,NUM_CATEGORIES-1),randint(0,NUM_CATEGORIES-1))} for _ in range(10000)]


In [40]:
def check(bag: set) -> bool:
    return any(i <= bag for i in INCOMPATIBLE)
# def cost(solution: list[set]) -> int:
#     return sum(check(b) for b in solution)

#for debugging
def cost(solution):
    # 代价 = 违规袋子的数量（越小越好；0 表示全可行）
    return sum(1 for bag in solution if not check(bag))


#试试其他的移动工具
# 统计某个袋子里当前有多少对冲突
def _bag_conf_pairs(bag):
    return sum(1 for a, b in INCOMPATIBLE if (a in bag and b in bag))

# 统计某个物品在该袋中引发的冲突度（它和袋内多少个物品构成不相容）
def _item_conf_degree(x, bag):
    return sum(1 for a, b in INCOMPATIBLE if (x == a and b in bag) or (x == b and a in bag))



In [41]:


#1.all mutation is possible
#2.mutation is not too big -- small change
#3.expolration vs exploitation -- different phase of the search
# def tweak(bag: list[set],t:float = 0.1) -> list[set]:
#     new_bags = deepcopy(bag)

#     again = True
#     while again:
#         # b1 = choice(i for i in range(NUM_ITEM) if check(new_bags[i]))
#         candidates = [i for i in range(NUM_ITEM) if invalid(new_bags[i])]
#         if not candidates:
#             # 没有任何满足 check 的袋子，直接返回或打破循环
#             return new_bags  # 或者：break

#         b1 = choice(candidates)
#         b2 = randint(0,NUM_ITEM-1)
#         b1_,b2_ = sorted(new_bags[b1]),sorted(new_bags[b2])
#         i = randint(0,NUM_CATEGORIES-1)
#         b1_[i],b2_[i] = b2_[i],b1_[i]
#         new_bags[b1],new_bags[b2] = set(b1_),set(b2_)
#         again = random() < t
#     return new_bags


#new tweak for debugging
# #but current_cost still not good, =50
# def tweak(current_solution, t=0.2):
#     # 逐袋 copy，避免浅拷贝
#     new_bags = [b.copy() for b in current_solution]

#     # 先构造列表；为空要兜底，否则 random.choice 会报错
#     candidates = [i for i in range(NUM_ITEM) if check(new_bags[i])]
#     if not candidates:
#         # 没有满足 check 的袋子，就从所有袋子随机挑两个不同的做扰动
#         i, j = sample(range(NUM_ITEM), 2)
#     else:
#         i = choice(candidates)
#         j = randrange(NUM_ITEM)
#         if i == j:
#             # 确保两个不同
#             j = (j + 1) % NUM_ITEM

   
#     # 从 i 袋拿一个放到 j 袋；如果本来是交换，就不变
#     if new_bags[i]:
#         itm = choice(list(new_bags[i]))
#         new_bags[i].remove(itm)
#         new_bags[j].add(itm)

#     return new_bags

#new tweak for debugging
def tweak(current_solution, t=0.2):
    # 逐袋 copy，避免浅拷贝
    new_bags = [b.copy() for b in current_solution]
    NB = len(new_bags)

    # 1) 先定位“有冲突”的袋子；没有的话直接返回（说明已可行）
    bad_bags = [i for i, b in enumerate(new_bags) if _bag_conf_pairs(b) > 0]
    if not bad_bags:
        return new_bags

    # 2) 从一个有冲突的袋子里，挑一个“冲突度最高”的物品作为移动对象
    i = choice(bad_bags)
    if not new_bags[i]:
        return new_bags
    itm = max(new_bags[i], key=lambda x: _item_conf_degree(x, new_bags[i]))

    # 3) 把它挪到“增量冲突最小”的目标袋
    best_j, best_inc = None, 10**9
    for j in range(NB):
        if j == i:
            continue
        inc = _item_conf_degree(itm, new_bags[j])
        if inc < best_inc:
            best_inc, best_j = inc, j

    if best_j is None:
        return new_bags

    new_bags[i].remove(itm)
    new_bags[best_j].add(itm)
    return new_bags

In [ ]:
current_solution = [{ITEMS[c][i] for c in range(NUM_CATEGORIES)} for i in range(NUM_ITEM)]
current_cost = cost(current_solution)

# ic(current_cost)

# max_steps = 100

# for steps in tqdm(range(1000)):
#     if current_cost == 0:
#         break
#     new_solution = tweak(current_solution,t=0.2)
#     new_cost = cost(new_solution)
#     if new_cost < current_cost:
#         current_solution = new_solution
#         current_cost = new_cost 
#         ic(current_cost)
# print(current_cost)

#use simulated annealing to solve the problem
steps_total = 1000
T = 1.0         # 初始温度
alpha = 0.995   # 冷却率

best_solution = [b.copy() for b in current_solution]
best_cost = cost(best_solution)

for steps in tqdm(range(steps_total)):
    if current_cost == 0:
        break

    new_solution = tweak(current_solution, t=0.2)
    new_cost = cost(new_solution)
    delta = new_cost - current_cost

    # 接受准则：改进必收；持平可收；恶化以概率收（帮助跳出局部最优/平台）
    accept = False
    if new_cost < current_cost:
        accept = True
    elif new_cost == current_cost:
        accept = True
    else:
        if T > 1e-8 and random.random() < exp(-delta / T):
            accept = True

    if accept:
        current_solution = new_solution
        current_cost = new_cost
        if current_cost < best_cost:
            best_cost = current_cost
            best_solution = [b.copy() for b in current_solution]
            ic(current_cost)  # 你原来的调试输出

    T *= alpha

# 用历史最优作为最终输出
current_solution, current_cost = best_solution, best_cost
print(current_cost)

  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)